CONTAGION 
NODE BLOCKING
ERDOS-RENYI GRAPH


In [1]:
import matplotlib.pyplot as plt
from networkx import nx
import numpy as np
import algorithmx
import random
import pandas as pd
from collections import defaultdict

n = 100  # number of nodes
#m = int(((n*(n-1))/2)*(0.08)) # number of edges
m= 400

high_infected_percentage=0.15
medium_infected_percentage=0.1
low_infected_percentage=0.05

high_risky_percentage=0.5
medium_risky_percentage=0.3
low_risky_percentage=0.1


infected_percentage=medium_infected_percentage
total_risky_percentage=1-infected_percentage
risky_percentage=medium_risky_percentage

total_infected=int(infected_percentage*n)
total_risky=int(risky_percentage*n)

G = nx.gnm_random_graph(n, m)

colnames=range(1,(n+1))
adjacency_df = pd.DataFrame(np.zeros((n,n)),columns=colnames,index=colnames)

for (u, v) in G.edges():
    G.edges[u,v]['weight'] = np.random.uniform(0,1)
    adjacency_df.iat[u,v]=G.edges[u,v]['weight']
    adjacency_df.iat[v,u]=G.edges[u,v]['weight']

#nx.draw(G)
#plt.show()
dummy_vector=np.arange(1,n+1)
dummy_selected=np.random.choice(dummy_vector,(total_infected+total_risky),replace=False)
selected_infected=np.random.choice(dummy_selected,total_infected,replace=False)
selected_risky= [i for i in dummy_selected if i not in selected_infected]

adjacency_df['Threshold']=np.random.uniform(0,1,size=n) #Tekrar bak ...
adjacency_df['IsInfected']=np.zeros((n,1))
adjacency_df['IsRisky']=np.zeros((n,1))
adjacency_df.loc[selected_infected,'IsInfected']=1
adjacency_df.loc[selected_risky,'IsRisky']=1

adjacency_df.to_csv("adjacency_df-n"+str(n)+'-m'+str(m)+".csv",index=False)

#sets
setV = range(1,n+1)


In [2]:
(adjacency_df['IsRisky']==1).sum()

30

In [3]:
impact_matrix = adjacency_df.copy(deep=True)
impact_matrix.loc[:,setV]=impact_matrix.loc[:,setV].div(impact_matrix.loc[:,"Threshold"],axis=0)
impact_matrix[impact_matrix[setV] >= 1] = 1

In [4]:
impact_matrix

,1,2,3,4,5,6,7,8,9,10,...,94,95,96,97,98,99,100,Threshold,IsInfected,IsRisky
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.344463,1.0,0.0
2,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.425826,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.248227,0.0,0.0
4,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,1.000000,0.0,0.0,1.000000,0.135131,0.0,0.0
5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,1.0,0.000000,0.244292,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.000000,0.0,...,0.000000,1.0,0.0,0.000000,0.0,0.0,0.000000,0.510686,0.0,0.0
97,0.0,0.0,0.0,0.928886,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.632456,0.443785,0.0,0.0
98,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.640955,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.931519,0.0,1.0
99,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.000000,0.0,...,0.428771,0.0,0.0,0.000000,0.0,0.0,0.000000,0.603154,0.0,0.0


In [5]:
Di_G = nx.DiGraph(G)
for (u, v) in G.edges():
    Di_G.edges[u,v]['weight'] = impact_matrix.iat[v,u]
    Di_G.edges[v,u]['weight'] = impact_matrix.iat[u,v]

In [6]:
node_features=pd.DataFrame(np.zeros((n,1)),columns=["IsInfected"],index=range(1,n+1))

In [7]:
betweenness_cent=nx.betweenness_centrality(Di_G)
degree_cent=nx.degree_centrality(Di_G)
closeness_cent=nx.closeness_centrality(Di_G)
eigenvector_cent=nx.eigenvector_centrality(Di_G)
degree=Di_G.degree()

In [8]:
node_features['IsInfected']=adjacency_df['IsInfected']
node_features['IsRisky']=adjacency_df['IsRisky']
node_features['map_for_dict']=betweenness_cent
node_features['Betweenness']=node_features['map_for_dict'].map(betweenness_cent)
node_features['Degree_Cent']=node_features['map_for_dict'].map(degree_cent)
node_features['Closeness_cent']=node_features['map_for_dict'].map(closeness_cent)
node_features['Eigenvector_cent']=node_features['map_for_dict'].map(eigenvector_cent)
node_features['Degree']=node_features['map_for_dict'].map(degree)

In [9]:
#sets
setS = node_features.loc[(node_features.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = node_features.loc[(node_features.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()
setRisky= node_features.loc[(node_features.loc[:, "IsRisky"] > 0),"IsRisky"].index.tolist()

In [10]:
node_features['Shortest_to_Infected']=9999
node_features['Shortest_to_Risky']=0
for i in setS:
    shortest_inf=np.array([])
    shortest_risky=np.array([])
    for j in setP:
        shortest_inf=np.append(shortest_inf,nx.shortest_path_length(Di_G,source=i-1,target=j-1))
    node_features.at[i,'Shortest_to_Infected']=shortest_inf.min()
    for k in setRisky:
        shortest_risky=np.append(shortest_risky,nx.shortest_path_length(Di_G,source=i-1,target=k-1))
    node_features.at[i,'Shortest_to_Risky']=shortest_risky.min()

In [11]:
node_features['Total_Impact_Ratio']=impact_matrix[setV].sum(axis=1)

In [12]:
node_features.describe()

,IsInfected,IsRisky,map_for_dict,Betweenness,Degree_Cent,Closeness_cent,Eigenvector_cent,Degree,Shortest_to_Infected,Shortest_to_Risky,Total_Impact_Ratio
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.100000,0.300000,49.500000,0.014442,0.161616,0.416117,0.094306,16.000000,1001.180000,0.630000,6.194179
std,0.301511,0.460566,29.011492,0.010198,0.055604,0.029345,0.033429,5.504819,3014.383155,0.544115,2.490285
min,0.000000,0.000000,0.000000,0.000573,0.040404,0.331104,0.025712,4.000000,1.000000,0.000000,0.985422
25%,0.000000,0.000000,24.750000,0.006544,0.121212,0.395606,0.068612,12.000000,1.000000,0.000000,4.397402
50%,0.000000,0.000000,49.500000,0.013066,0.161616,0.419492,0.093979,16.000000,1.000000,1.000000,6.000000
75%,0.000000,1.000000,74.250000,0.020188,0.202020,0.434211,0.116109,20.000000,2.000000,1.000000,7.760081
max,1.000000,1.000000,99.000000,0.049718,0.303030,0.478261,0.184673,30.000000,9999.000000,2.000000,13.879576


In [13]:
node_features.to_csv("node_features-n"+str(n)+'-m'+str(m)+".csv",index=False)